In [95]:
import math
import copy
import time
import numpy as np

from pprint import pprint
from tqdm import tqdm

In [106]:
class SudokuSolver:
    def __init__(self, board):
        self.initial_board = board
        self.solution_board = copy.deepcopy(board)
        self.num_rows = len(board)
        self.num_cols = len(board[0])
        self.sq_rt = int(math.sqrt(self.num_rows))
        
    def is_valid(self, ith_row, jth_col, num):
        return (
        all(num != self.solution_board[ith_row][i] for i in range(self.num_rows)) and # horizontal constraint
        all(num != self.solution_board[i][jth_col] for i in range(self.num_cols)) and # vertical constraint
        all(num != self.solution_board[ith_row // self.sq_rt * self.sq_rt + i][jth_col // self.sq_rt * self.sq_rt + j] for i in range(self.sq_rt) for j in range(self.sq_rt)) # within local cell constraint
        )
    
    def find_empty_location(self):
        # Find the first empty cell in the board (marked with 0)
        for i in range(self.num_rows):
            for j in range(self.num_cols):
                if self.solution_board[i][j] == 0:
                    return i, j
        return None, None
    
    def solve_sudoku(self):
        start_t = time.time()
        if self.backtrack_sudoku():
            end_t = time.time()
#             print("Total time taken: ", end_t - start_t)
            return self.solution_board
        else:
            end_t = time.time()
#             print("Total time taken: ", end_t - start_t)
            print("Didn't solve the given sudoku puzzle!")
            return self.initial_board
    
    def backtrack_sudoku(self):
        # empty_location
        ith_row, jth_col = self.find_empty_location()

        # if there is no empty location, the sudoku is solved
        if ith_row is None:
            return True # base case here
        
        # domain for this particular applicaiton is [1 to num_rows]
        for num in range(1, self.num_rows + 1):
            
            if self.is_valid(ith_row, jth_col, num):
                self.solution_board[ith_row][jth_col] = num
                
                # recursively try to fill the sudoku solution board
                if self.backtrack_sudoku():
                    return True

                # If the current assignment leads to an invalid solution, backtrack
                self.solution_board[ith_row][jth_col] = 0

        return False

In [96]:
boards = np.zeros((1000000, 81), np.int32)
solutions = np.zeros((1000000, 81), np.int32)
for i, line in enumerate(open('sudoku.csv', 'r').read().splitlines()[1:]):
    board, solution = line.split(",")
    for j, q_s in enumerate(zip(board, solution)):
        q, s = q_s
        boards[i, j] = q
        solutions[i, j] = s
boards = boards.reshape((-1, 9, 9))
solutions = solutions.reshape((-1, 9, 9))

In [100]:
def calculate_accuracy(true_array, predicted_array):
    if len(true_array) != len(predicted_array) or any(len(row) != len(predicted_array[0]) for row in true_array):
        raise ValueError("Input arrays must have the same dimensions")

    total_elements = len(true_array) * len(true_array[0])
    correct_predictions = sum(true_array[i][j] == predicted_array[i][j] for i in range(len(true_array)) for j in range(len(true_array[0])))

    accuracy = correct_predictions / total_elements
    return accuracy

In [112]:
all_accuracies = []
for i in tqdm(range(30000), ncols = 120):
    board = boards[i]
    solution = solutions[i]
    
    sudoku = SudokuSolver(board)
    sudoku_solution = sudoku.solve_sudoku()
    all_accuracies.append(calculate_accuracy(solution, sudoku_solution))

100%|████████████████████████████████████████████████████████████████████████████| 30000/30000 [03:23<00:00, 147.35it/s]


In [82]:
sudoku_board = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 5, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 6, 0],
    [0, 0, 0, 0, 6, 0, 0, 0, 3],
    [0, 0, 0, 8, 0, 3, 0, 0, 1],
    [0, 0, 0, 0, 2, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 0, 0, 0, 0, 0]
]
start_t = time.time()
sudoku = SudokuSolver(sudoku_board)
solution = sudoku.solve_sudoku()
pprint(solution)
end_t = time.time()
end_t - start_t

Total time taken:  0.02397441864013672
[[1, 2, 3, 6, 4, 7, 5, 9, 8],
 [4, 6, 7, 9, 8, 5, 1, 3, 2],
 [5, 8, 9, 1, 3, 2, 4, 6, 7],
 [2, 7, 8, 5, 6, 1, 9, 4, 3],
 [6, 4, 5, 8, 9, 3, 7, 2, 1],
 [3, 9, 1, 7, 2, 4, 8, 5, 6],
 [7, 1, 4, 3, 5, 6, 2, 8, 9],
 [8, 3, 2, 4, 1, 9, 6, 7, 5],
 [9, 5, 6, 2, 7, 8, 3, 1, 4]]


0.024930238723754883

In [99]:


# Example usage:
true_array = [
    [1, 0, 1],
    [0, 1, 0],
    [1, 1, 1]
]

predicted_array = [
    [1, 0, 0],
    [0, 1, 0],
    [1, 1, 1]
]

accuracy = calculate_accuracy(true_array, predicted_array)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 88.89%
